In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("D:/data/Aaron/ProstateMRL/Code/Extraction/PatientKey.csv")

df = df[df["Treatment"] == "20fractions"]
list = df["PatID"].unique().astype(str)
array = np.asarray(list)
print(array)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/data/Aaron/ProstateMRL/Code/Extraction/PatientKey.csv'

In [2]:
import pandas as pd
import numpy as np
import datetime

import sys
import os

current = os.path.dirname(os.path.realpath("Test.ipynb"))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
import ImageFunctions as IF

In [4]:
# read in patient key
root = UF.DataRoot(2)
LimbusKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey.csv")
AllKey = pd.read_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\AllPatientKey_s.csv")

# Loop through treatments
treatments = AllKey["Treatment"].unique()
new_key = pd.DataFrame()
for t in treatments:
    AllKey_t = AllKey[AllKey["Treatment"] == t]
    LimbusKey_t = LimbusKey[LimbusKey["Treatment"] == t]

    # Loop through patients
    patIDs = LimbusKey_t["PatID"].unique()

    for pat in patIDs:
        pat_L = LimbusKey_t[LimbusKey_t["PatID"] == pat]
        pat_A = AllKey_t[AllKey_t["PatID"] == pat]
        print(pat_A)
        print(pat_L)
        # add date and days column from all to limbus key based on Scan
        for i in range(len(pat_L)):
            scan = pat_L.iloc[i]["Scan"]
            fraction = pat_A.iloc[i]["Fraction"]
            date = pat_A[pat_A["Scan"] == scan]["Date"].values[0]
            days = pat_A[pat_A["Scan"] == scan]["Days"].values[0]
            pat_L.at[pat_L.index[i], "Date"] = str(date)
            pat_L.at[pat_L.index[i], "Days"] = int(days)
            pat_L.at[pat_L.index[i], "Fraction"] = int(fraction)

        pat_L["Days"] = pat_L["Days"].astype(int)
        # sort by days
        pat_L = pat_L.sort_values(by=["Fraction"])

    # add to new key
        new_key = new_key.append(pat_L)

new_key.to_csv(root + "\\Aaron\\ProstateMRL\\Code\\Extraction\\PatKeys\\LimbusKey_s.csv", index=False)


        
        



   PatID    Treatment      FileDir  Scan      Date  Days  Fraction
0    213  20fractions  20fractions   MR4  20110330     0         1
1    213  20fractions  20fractions  MR17  20110403     4         2
2    213  20fractions  20fractions  MR37  20110413    14         3
3    213  20fractions  20fractions  MR52  20110420    21         4
4    213  20fractions  20fractions  MR66  20110424    25         5
     Directory    Treatment  PatID  Scan
0  20fractions  20fractions    213  MR17
1  20fractions  20fractions    213  MR37
2  20fractions  20fractions    213   MR4
3  20fractions  20fractions    213  MR52
4  20fractions  20fractions    213  MR66
   PatID    Treatment      FileDir  Scan      Date  Days  Fraction
5    231  20fractions  20fractions   MR3  20110316     0         1
6    231  20fractions  20fractions  MR19  20110320     4         2
7    231  20fractions  20fractions  MR34  20110327    11         3
8    231  20fractions  20fractions  MR49  20110403    18         4
9    231  20fract

c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is 

    PatID    Treatment          FileDir Scan      Date  Days  Fraction
90    397  20fractions  20fractions_new  MR1  20110622     0         1
91    397  20fractions  20fractions_new  MR2  20110626     4         2
92    397  20fractions  20fractions_new  MR3  20110703    11         3
93    397  20fractions  20fractions_new  MR4  20110710    18         4
94    397  20fractions  20fractions_new  MR5  20110717    25         5
          Directory    Treatment  PatID Scan
90  20fractions_new  20fractions    397  MR1
91  20fractions_new  20fractions    397  MR2
92  20fractions_new  20fractions    397  MR3
93  20fractions_new  20fractions    397  MR4
94  20fractions_new  20fractions    397  MR5
    PatID    Treatment          FileDir Scan      Date  Days  Fraction
95    894  20fractions  20fractions_new  MR1  20120225     0         1
96    894  20fractions  20fractions_new  MR2  20120302     6         2
97    894  20fractions  20fractions_new  MR3  20120309    13         3
98    894  20fractio

c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a 

     PatID Treatment   FileDir Scan      Date  Days  Fraction
183   1642      SABR  SABR_new  MR1  20121018     0         1
184   1642      SABR  SABR_new  MR2  20121020     2         2
185   1642      SABR  SABR_new  MR3  20121025     7         3
186   1642      SABR  SABR_new  MR4  20121027     9         4
187   1642      SABR  SABR_new  MR5  20121101    14         5
    Directory Treatment  PatID Scan
183  SABR_new      SABR   1642  MR1
184  SABR_new      SABR   1642  MR2
185  SABR_new      SABR   1642  MR3
186  SABR_new      SABR   1642  MR4
187  SABR_new      SABR   1642  MR5
     PatID Treatment   FileDir  Scan      Date  Days  Fraction
188    829      SABR  SABR_new   MR1  20120210     0         1
189    829      SABR  SABR_new   MR5  20120212     2         2
190    829      SABR  SABR_new   MR8  20120215     5         3
191    829      SABR  SABR_new  MR11  20120217     7         4
192    829      SABR  SABR_new  MR15  20120219     9         5
    Directory Treatment  PatID  Sc

c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\b01297ar\Documents\ProstateMRL-local\py-envs\PR-env\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pingouin as pg
from scipy import stats

import sys

current = os.path.dirname(os.path.realpath(''))
parent = os.path.dirname(current)
sys.path.append(parent + "\\Functions\\")
import UsefulFunctions as UF
#import ImageFunctions as IF

root = UF.DataRoot(2)

# load in limbus data Limbus key
df_key = pd.read_csv(root + "Aaron\ProstateMRL\Code\Extraction\PatKeys\LimbusKey_s.csv")

# load in patient data
df_all = pd.read_csv(root + "Aaron\ProstateMRL\Data\Paper1\Limbus\Limbus_fts.csv")

# filter only SABR patients
patIDs = UF.SABRPats()[0:10] 
df_key = df_key[df_key["Treatment"] == "SABR"]
df_key = df_key[df_key["PatID"].isin(patIDs)]

# merge with limbus key, only want Days Column
df_all = pd.merge(df_all, df_key[["PatID", "Scan", "Days"]], on = ["PatID", "Scan"], how = "left")
# loop through all patients
df_final = pd.DataFrame()

df_all = df_all.drop(columns = [col for col in df_all.columns if "diagnostics" in col])
df_all = df_all.drop(columns = [col for col in df_all.columns if "Unnamed" in col])


# for pat in patIDs:
#     df_pat = df_all[df_all["PatID"].isin([pat])]

    # drop all columns with Diagnostic in name

    # df_m = df_pat[df_pat["Mask"] == "RP"]
    # df_l = df_pat[df_pat["Mask"] == "Limbus"]
    # # drop mask column
    # df_m = df_m.drop(columns = ["Mask"])
    # df_l = df_l.drop(columns = ["Mask"])

df_res = pd.DataFrame()

fts = df_all.columns[3:-1]

# loop through days
days = df_all["Days"].unique()
for day in days:
    df_day = df_all[df_all["Days"] == day]
    print(df_day)
    df_day = df_day.drop(columns = ["Scan", "Days"])
    print(df_day)

    # pivot df_day so that id is mask and variable is feature
    df_ft = df_day.melt(id_vars = ["Mask", "PatID"], var_name = "variable", value_name = "value")

    print(df_ft)

    fts = df_ft["variable"].unique()
    for ft in fts:
        df_ft = df_ft[df_ft["variable"] == ft]
        icc = pg.intraclass_corr(data = df_ft, targets = "PatID", raters = "mask", ratings = "value")
        print(icc)
        df_res = df_res.append(icc)


    PatID  Scan    Mask  original_firstorder_10Percentile  \
0     653  MR15      RP                              29.0   
1     653  MR15  Limbus                              33.0   
10    713  MR11      RP                              18.0   
11    713  MR11  Limbus                              27.0   
20    752  MR12      RP                              63.0   
21    752  MR12  Limbus                              66.0   
28    826  MR11      RP                              23.0   
29    826  MR11  Limbus                              22.0   
42   1088  MR19      RP                              10.0   
43   1088  MR19  Limbus                               8.0   
48   1089  MR15      RP                              17.0   
49   1089  MR15  Limbus                              19.0   
62   1303  MR13      RP                              20.0   
63   1303  MR13  Limbus                              20.0   
72   1307  MR14      RP                              33.0   
73   1307  MR14  Limbus 

AssertionError: 